In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import norm



In [4]:
ticker = "RELIANCE.NS"

data = yf.download(
    ticker,
    start="2018-01-01",
    end="2024-01-01",
    progress=False
)

prices = data["Close"].dropna()
prices.head()


/tmp/ipython-input-4244240006.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(


Ticker,RELIANCE.NS
Date,
2018-01-01,401.864746
2018-01-02,402.483215
2018-01-03,404.095520
2018-01-04,406.525055
2018-01-05,407.828186


In [5]:
returns = np.log(prices / prices.shift(1)).dropna()

In [6]:
sigma = returns.std() * np.sqrt(252)
sigma

,0
Ticker,
RELIANCE.NS,0.29742


In [7]:
S0 = prices.iloc[-1]   # Current stock price
K = S0                # At-the-money strike
T = 0.5               # 6 months to maturity
r = 0.06              # 6% risk-free rate (India)

In [8]:
S0

,2023-12-29
Ticker,
RELIANCE.NS,1282.979126


In [9]:
def black_scholes_call(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)
    return call_price


In [10]:
call_price = black_scholes_call(S0, K, T, r, sigma)
call_price

,2023-12-29
Ticker,
RELIANCE.NS,125.886899


In [11]:
def bs_delta_call(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    return norm.cdf(d1)

In [12]:
delta = bs_delta_call(S0, K, T, r, sigma)
delta

array([0.5978562])

In [13]:
def bs_gamma(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    return gamma

In [14]:
gamma = bs_gamma(S0, K, T, r, sigma)
gamma

,0
Ticker,
RELIANCE.NS,0.001434


In [15]:
def bs_vega(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    vega = S * norm.pdf(d1) * np.sqrt(T)
    return vega

In [16]:
vega = bs_vega(S0, K, T, r, sigma)
vega

,2023-12-29
Ticker,
RELIANCE.NS,350.97853


In [25]:
delta=delta[0]

IndexError: invalid index to scalar variable.

In [26]:
delta

np.float64(0.5978562019361237)

In [30]:
gamma=gamma[0]

/tmp/ipython-input-4173333276.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  gamma=gamma[0]


In [31]:
vega=vega[0]

/tmp/ipython-input-3314690176.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  vega=vega[0]


In [32]:
risk_table = pd.DataFrame({
    "Greek": ["Delta", "Gamma", "Vega"],
    "Value": [delta, gamma, vega],
    "Risk Interpretation": [
        "Directional stock price risk",
        "Hedging instability / curvature risk",
        "Volatility risk (uncertainty risk)"
    ]
})

risk_table


,Greek,Value,Risk Interpretation
0,Delta,0.597856,Directional stock price risk
1,Gamma,0.001434,Hedging instability / curvature risk
2,Vega,350.978530,Volatility risk (uncertainty risk)
